In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score, recall_score
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
df_big_data_bowl = pd.read_csv(r"C:\Users\davep\Downloads\plays.csv")
df_nfl_verse = pd.read_csv(r"C:\Users\davep\Downloads\play_by_play_2022.csv")

C:\Users\davep\AppData\Local\Temp\ipykernel_50636\2936651633.py:2: DtypeWarning: Columns (45,179,180,182,183,187,188,189,190,193,194,197,198,203,204,205,206,213,214,218,219,220,222,224,226,243,244,245,248,249,253,254,255,260,262,263,266,267,268,269,283,284) have mixed types. Specify dtype option on import or set low_memory=False.
  df_nfl_verse = pd.read_csv(r"C:\Users\davep\Downloads\play_by_play_2022.csv")


In [4]:
df_nfl_verse

,play_id,game_id,old_game_id,home_team,away_team,season_type,week,posteam,posteam_type,defteam,side_of_field,yardline_100,game_date,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,game_half,quarter_end,drive,sp,qtr,down,goal_to_go,time,yrdln,ydstogo,ydsnet,desc,play_type,yards_gained,shotgun,no_huddle,qb_dropback,qb_kneel,qb_spike,qb_scramble,pass_length,pass_location,air_yards,yards_after_catch,run_location,run_gap,field_goal_result,kick_distance,extra_point_result,two_point_conv_result,home_timeouts_remaining,away_timeouts_remaining,timeout,timeout_team,td_team,td_player_name,td_player_id,posteam_timeouts_remaining,defteam_timeouts_remaining,total_home_score,total_away_score,posteam_score,defteam_score,score_differential,posteam_score_post,defteam_score_post,score_differential_post,no_score_prob,opp_fg_prob,opp_safety_prob,opp_td_prob,fg_prob,safety_prob,td_prob,extra_point_prob,two_point_conversion_prob,ep,epa,total_home_epa,total_away_epa,total_home_rush_epa,total_away_rush_epa,total_home_pass_epa,total_away_pass_epa,air_epa,yac_epa,comp_air_epa,comp_yac_epa,total_home_comp_air_epa,total_away_comp_air_epa,total_home_comp_yac_epa,total_away_comp_yac_epa,total_home_raw_air_epa,total_away_raw_air_epa,total_home_raw_yac_epa,total_away_raw_yac_epa,wp,def_wp,home_wp,away_wp,wpa,vegas_wpa,vegas_home_wpa,home_wp_post,away_wp_post,vegas_wp,vegas_home_wp,total_home_rush_wpa,total_away_rush_wpa,total_home_pass_wpa,total_away_pass_wpa,air_wpa,yac_wpa,comp_air_wpa,comp_yac_wpa,total_home_comp_air_wpa,total_away_comp_air_wpa,total_home_comp_yac_wpa,total_away_comp_yac_wpa,total_home_raw_air_wpa,total_away_raw_air_wpa,total_home_raw_yac_wpa,total_away_raw_yac_wpa,punt_blocked,first_down_rush,first_down_pass,first_down_penalty,third_down_converted,third_down_failed,fourth_down_converted,fourth_down_failed,incomplete_pass,touchback,interception,punt_inside_twenty,punt_in_endzone,punt_out_of_bounds,punt_downed,punt_fair_catch,kickoff_inside_twenty,kickoff_in_endzone,kickoff_out_of_bounds,kickoff_downed,kickoff_fair_catch,fumble_forced,fumble_not_forced,fumble_out_of_bounds,solo_tackle,safety,penalty,tackled_for_loss,fumble_lost,own_kickoff_recovery,own_kickoff_recovery_td,qb_hit,rush_attempt,pass_attempt,sack,touchdown,pass_touchdown,rush_touchdown,return_touchdown,extra_point_attempt,two_point_attempt,field_goal_attempt,kickoff_attempt,punt_attempt,fumble,complete_pass,assist_tackle,lateral_reception,lateral_rush,lateral_return,lateral_recovery,passer_player_id,passer_player_name,passing_yards,receiver_player_id,receiver_player_name,receiving_yards,rusher_player_id,rusher_player_name,rushing_yards,lateral_receiver_player_id,lateral_receiver_player_name,lateral_receiving_yards,lateral_rusher_player_id,lateral_rusher_player_name,lateral_rushing_yards,lateral_sack_player_id,lateral_sack_player_name,interception_player_id,interception_player_name,lateral_interception_player_id,lateral_interception_player_name,punt_returner_player_id,punt_returner_player_name,lateral_punt_returner_player_id,lateral_punt_returner_player_name,kickoff_returner_player_name,kickoff_returner_player_id,lateral_kickoff_returner_player_id,lateral_kickoff_returner_player_name,punter_player_id,punter_player_name,kicker_player_name,kicker_player_id,own_kickoff_recovery_player_id,own_kickoff_recovery_player_name,blocked_player_id,blocked_player_name,tackle_for_loss_1_player_id,tackle_for_loss_1_player_name,tackle_for_loss_2_player_id,tackle_for_loss_2_player_name,qb_hit_1_player_id,qb_hit_1_player_name,qb_hit_2_player_id,qb_hit_2_player_name,forced_fumble_player_1_team,forced_fumble_player_1_player_id,forced_fumble_player_1_player_name,forced_fumble_player_2_team,forced_fumble_player_2_player_id,forced_fumble_player_2_player_name,solo_tackle_1_team,solo_tackle_2_team,solo_tackle_1_player_id,solo_tackle_2_player_id,solo_tackle_1_player_name,solo_tackle_2_player_name,assist_tackle_1_player_id,assist_tackle_1_player_name,assist_tackle_1_team,assist_tack

In [5]:
df_big_data_bowl

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,playClockAtSnap,passResult,passLength,targetX,targetY,playAction,dropbackType,dropbackDistance,passLocationType,timeToThrow,timeInTackleBox,timeToSack,passTippedAtLine,unblockedPressure,qbSpike,qbKneel,qbSneak,rushLocationType,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,01:54,35,17,N,31,0.982017,0.017983,0.719313,EMPTY,3x2,10.0,C,6.0,36.69,16.51,False,TRADITIONAL,2.40,INSIDE_BOX,2.990,2.990,NaN,False,False,False,0,NaN,NaN,NaN,9,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone
1,2022091809,3698,(2:13) (Shotgun) J.Burrow pass short right to ...,4,1,10,CIN,DAL,CIN,8,02:13,17,17,N,18,0.424356,0.575644,0.607746,EMPTY,3x2,9.0,C,4.0,20.83,20.49,False,TRADITIONAL,1.14,INSIDE_BOX,1.836,1.836,NaN,False,False,False,0,NaN,NaN,NaN,4,4,0.002847,-0.002847,-0.240509,True,NaN,NaN,0,Quarters,Zone
2,2022103004,3146,(2:00) (Shotgun) D.Mills pass short right to D...,4,3,12,HOU,TEN,HOU,20,02:00,3,17,N,30,0.006291,0.993709,-0.291485,SHOTGUN,2x2,12.0,C,-4.0,26.02,17.56,False,TRADITIONAL,3.20,INSIDE_BOX,2.236,2.236,NaN,False,False,False,0,NaN,NaN,NaN,6,6,0.000205,-0.000205,-0.218480,True,NaN,NaN,0,Quarters,Zone
3,2022110610,348,(9:28) (Shotgun) P.Mahomes pass short left to ...,1,2,10,KC,TEN,TEN,23,09:28,0,0,N,33,0.884223,0.115777,4.249382,SHOTGUN,2x2,11.0,C,-6.0,38.95,14.19,False,TRADITIONAL,3.02,INSIDE_BOX,2.202,2.202,NaN,False,False,False,0,NaN,NaN,NaN,4,4,-0.001308,0.001308,-0.427749,True,NaN,NaN,0,Quarters,Zone
4,2022102700,2799,(2:16) (Shotgun) L.Jackson up the middle to TB...,3,2,8,BAL,TB,TB,27,02:16,10,10,N,37,0.410371,0.589629,3.928413,PISTOL,3x1,8.0,NaN,NaN,NaN,NaN,True,DESIGNED_RUN,2.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,False,INSIDE_LEFT,NaN,-1,-1,0.027141,-0.027141,-0.638912,False,MAN,READ OPTION,0,Cover-1,Man
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16119,2022110604,1051,(12:49) (Shotgun) T.Lawrence pass incomplete s...,2,3,4,JAX,LV,JAX,31,12:49,0,10,N,79,0.207444,0.792556,0.702276,SHOTGUN,2x2,9.0,I,6.0,72.69,39.92,False,TRADITIONAL,1.67,INSIDE_BOX,1.860,1.860,NaN,False,False,False,0,NaN,NaN,NaN,0,0,-0.024741,0.024741,-1.391687,True,NaN,NaN,0,Cover-2,Zone
16120,2022103005,3492,(12:32) (Shotgun) K.Cousins pass incomplete de...,4,1,10,MIN,ARI,MIN,25,12:32,28,26,N,35,0.724565,0.275435,1.600202,SHOTGUN,2x2,12.0,I,20.0,NaN,NaN,False,TRADITIONAL,2.24,INSIDE_BOX,3.136,3.136,NaN,False,False,False,0,NaN,NaN,NaN,0,0,-0.026580,0.026580,-0.503208,True,NaN,NaN,0,Cover-3,Zone
16121,2022092502,2337,(9:59) (Shotgun) P.Mahomes scrambles right end...,3,1,10,KC,IND,IND,13,09:59,10,14,N,97,0.165774,0.834226,5.127508,SHOTGUN,2x2,6.0,R,NaN,NaN,NaN,False,SCRAMBLE,3.28,NaN,NaN,NaN,NaN,NaN,False,NaN,0,False,INSIDE_RIGHT,NaN,10,10,-0.013790,0.013790,1.073898,True,NaN,NaN,0,Quarters,Zone
16122,2022091809,719,(:45) C.Rush pass incomplete deep right to C.L...,1,1,10,DAL,CIN,CIN,47,00:45,7,3,N,57,0.519735,0.480265,2.926312,SINGLEBACK,2x2,3.0,I,38.0,21.89,49.48,True,TRADITIONAL,6.64,INSIDE_BOX,3.070,3.070,NaN,False,False,False,0,NaN,NaN,NaN,0,0,-0.011561,0.011561,-0.522397,True,UNDEFINED,NaN,0,Cover-3,Zone


In [6]:
df_big_data_bowl[(df_big_data_bowl["gameId"] == 2022102302) & (df_big_data_bowl["gameClock"] == "01:54")]

,gameId,playId,playDescription,quarter,down,yardsToGo,possessionTeam,defensiveTeam,yardlineSide,yardlineNumber,gameClock,preSnapHomeScore,preSnapVisitorScore,playNullifiedByPenalty,absoluteYardlineNumber,preSnapHomeTeamWinProbability,preSnapVisitorTeamWinProbability,expectedPoints,offenseFormation,receiverAlignment,playClockAtSnap,passResult,passLength,targetX,targetY,playAction,dropbackType,dropbackDistance,passLocationType,timeToThrow,timeInTackleBox,timeToSack,passTippedAtLine,unblockedPressure,qbSpike,qbKneel,qbSneak,rushLocationType,penaltyYards,prePenaltyYardsGained,yardsGained,homeTeamWinProbabilityAdded,visitorTeamWinProbilityAdded,expectedPointsAdded,isDropback,pff_runConceptPrimary,pff_runConceptSecondary,pff_runPassOption,pff_passCoverage,pff_manZone
0,2022102302,2655,(1:54) (Shotgun) J.Burrow pass short middle to...,3,1,10,CIN,ATL,CIN,21,01:54,35,17,N,31,0.982017,0.017983,0.719313,EMPTY,3x2,10.0,C,6.0,36.69,16.51,False,TRADITIONAL,2.4,INSIDE_BOX,2.99,2.99,NaN,False,False,False,0,NaN,NaN,NaN,9,9,0.004634,-0.004634,0.702717,True,NaN,NaN,0,Cover-3,Zone


In [7]:
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [8]:
df_big_data_bowl = df_big_data_bowl[[
    "quarter", "down", "yardsToGo", "possessionTeam",
    "defensiveTeam", "yardlineNumber",
    "preSnapHomeScore", "preSnapVisitorScore", 
    "offenseFormation", "playClockAtSnap", "rushLocationType", "gameId", "playId"
]]

In [9]:
df_nfl_verse = df_nfl_verse[["home_team", "away_team", "quarter_seconds_remaining", 
                            "down", "goal_to_go", "ydstogo", "play_type", "pass_length", "pass_location",
                            "shotgun", "qb_dropback", "qb_kneel", "qb_spike", "qb_scramble", "run_location",
                            "run_gap", "old_game_id", "play_id", "qtr", "posteam"]]

In [10]:
merge_columns_big_data_bowl = ["gameId", "playId", "quarter", "down", "yardsToGo", "possessionTeam"]

In [11]:
merge_columns_nfl_verse = ["old_game_id", "play_id", "qtr", "down", "ydstogo", "posteam"]

In [12]:
merged = df_nfl_verse.merge(
    df_big_data_bowl, 
    left_on=merge_columns_nfl_verse, 
    right_on=merge_columns_big_data_bowl, 
    how="inner"
)

In [13]:
merged

,home_team,away_team,quarter_seconds_remaining,down,goal_to_go,ydstogo,play_type,pass_length,pass_location,shotgun,qb_dropback,qb_kneel,qb_spike,qb_scramble,run_location,run_gap,old_game_id,play_id,qtr,posteam,quarter,yardsToGo,possessionTeam,defensiveTeam,yardlineNumber,preSnapHomeScore,preSnapVisitorScore,offenseFormation,playClockAtSnap,rushLocationType,gameId,playId
0,NYJ,BAL,896,1.0,0,10,run,NaN,NaN,0,0.0,0,0,0,left,end,2022091107,68,1,NYJ,1,10,NYJ,BAL,22,0,0,SINGLEBACK,14.0,OUTSIDE_LEFT,2022091107,68
1,NYJ,BAL,865,2.0,0,10,run,NaN,NaN,0,0.0,0,0,0,left,end,2022091107,115,1,NYJ,1,10,NYJ,BAL,41,0,0,SINGLEBACK,24.0,INSIDE_RIGHT,2022091107,115
2,NYJ,BAL,841,3.0,0,5,pass,short,right,1,1.0,0,0,0,NaN,NaN,2022091107,136,1,NYJ,1,5,NYJ,BAL,46,0,0,SHOTGUN,25.0,NaN,2022091107,136
3,NYJ,BAL,822,1.0,0,10,pass,short,right,0,1.0,0,0,0,NaN,NaN,2022091107,202,1,BAL,1,10,BAL,NYJ,28,0,0,SINGLEBACK,14.0,NaN,2022091107,202
4,NYJ,BAL,801,2.0,0,6,pass,short,left,1,1.0,0,0,0,NaN,NaN,2022091107,230,1,BAL,1,6,BAL,NYJ,32,0,0,SHOTGUN,25.0,NaN,2022091107,230
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16119,KC,TEN,256,3.0,0,10,pass,short,right,1,1.0,0,0,0,NaN,NaN,2022110610,4525,5,KC,5,10,KC,TEN,11,17,17,SHOTGUN,9.0,NaN,2022110610,4525
16120,KC,TEN,244,1.0,0,10,run,NaN,NaN,0,0.0,0,0,0,right,guard,2022110610,4581,5,TEN,5,10,TEN,KC,25,20,17,SINGLEBACK,9.0,INSIDE_RIGHT,2022110610,4581
16121,KC,TEN,206,2.0,0,9,pass,NaN,NaN,1,1.0,0,0,0,NaN,NaN,2022110610,4602,5,TEN,5,9,TEN,KC,26,20,17,PISTOL,9.0,NaN,2022110610,4602
16122,KC,TEN,159,3.0,0,21,pass,NaN,NaN,1,1.0,0,0,0,NaN,NaN,2022110610,4621,5,TEN,5,21,TEN,KC,14,20,17,SHOTGUN,5.0,NaN,2022110610,4621


In [14]:
merged = merged[["down", "ydstogo", "play_type", "qtr", "yardlineNumber", "quarter_seconds_remaining", "offenseFormation", 
                "playClockAtSnap"]]

In [15]:
merged

,down,ydstogo,play_type,qtr,yardlineNumber,quarter_seconds_remaining,offenseFormation,playClockAtSnap
0,1.0,10,run,1,22,896,SINGLEBACK,14.0
1,2.0,10,run,1,41,865,SINGLEBACK,24.0
2,3.0,5,pass,1,46,841,SHOTGUN,25.0
3,1.0,10,pass,1,28,822,SINGLEBACK,14.0
4,2.0,6,pass,1,32,801,SHOTGUN,25.0
...,...,...,...,...,...,...,...,...
16119,3.0,10,pass,5,11,256,SHOTGUN,9.0
16120,1.0,10,run,5,25,244,SINGLEBACK,9.0
16121,2.0,9,pass,5,26,206,PISTOL,9.0
16122,3.0,21,pass,5,14,159,SHOTGUN,5.0


In [16]:
for col in merged.columns:
    print(merged[col].isna().sum())

0
0
0
0
0
0
188
1


In [17]:
merged['offenseFormation'].value_counts()

offenseFormation
SHOTGUN       8791
SINGLEBACK    3915
EMPTY         1342
I_FORM        1035
PISTOL         641
JUMBO          125
WILDCAT         87
Name: count, dtype: int64

In [18]:
merged = merged.dropna(subset=["offenseFormation"])

In [19]:
merged["playClockAtSnap"].fillna(merged["playClockAtSnap"].mean(), inplace=True)

C:\Users\davep\AppData\Local\Temp\ipykernel_50636\3581260016.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  merged["playClockAtSnap"].fillna(merged["playClockAtSnap"].mean(), inplace=True)
C:\Users\davep\AppData\Local\Temp\ipykernel_50636\3581260016.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged["playClockAtSnap"].fillna(merge

In [20]:
merged

,down,ydstogo,play_type,qtr,yardlineNumber,quarter_seconds_remaining,offenseFormation,playClockAtSnap
0,1.0,10,run,1,22,896,SINGLEBACK,14.0
1,2.0,10,run,1,41,865,SINGLEBACK,24.0
2,3.0,5,pass,1,46,841,SHOTGUN,25.0
3,1.0,10,pass,1,28,822,SINGLEBACK,14.0
4,2.0,6,pass,1,32,801,SHOTGUN,25.0
...,...,...,...,...,...,...,...,...
16119,3.0,10,pass,5,11,256,SHOTGUN,9.0
16120,1.0,10,run,5,25,244,SINGLEBACK,9.0
16121,2.0,9,pass,5,26,206,PISTOL,9.0
16122,3.0,21,pass,5,14,159,SHOTGUN,5.0


In [21]:
merged["qtr"].value_counts()

qtr
2    4336
4    4268
1    3602
3    3597
5     133
Name: count, dtype: int64

In [22]:
y = merged["play_type"]
X = merged.drop(columns=["play_type"])

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [24]:
X_train

,down,ydstogo,qtr,yardlineNumber,quarter_seconds_remaining,offenseFormation,playClockAtSnap
1687,2.0,1,4,49,64,SHOTGUN,30.0
7867,2.0,9,3,38,535,SHOTGUN,7.0
7231,1.0,10,1,39,341,SINGLEBACK,11.0
2615,3.0,7,4,38,637,SHOTGUN,6.0
11424,3.0,20,2,50,43,SHOTGUN,4.0
...,...,...,...,...,...,...,...
13583,1.0,10,1,39,527,SINGLEBACK,8.0
5458,4.0,22,4,26,541,SHOTGUN,6.0
872,1.0,10,1,43,385,SINGLEBACK,12.0
15980,2.0,6,4,6,215,SHOTGUN,7.0


In [25]:
y_train.value_counts()

play_type
pass    7475
run     5273
Name: count, dtype: int64

In [26]:
X_train["total_seconds_remaining"] = (4 - X_train["qtr"]) * 900 + X_train["quarter_seconds_remaining"]

In [27]:
X_train

,down,ydstogo,qtr,yardlineNumber,quarter_seconds_remaining,offenseFormation,playClockAtSnap,total_seconds_remaining
1687,2.0,1,4,49,64,SHOTGUN,30.0,64
7867,2.0,9,3,38,535,SHOTGUN,7.0,1435
7231,1.0,10,1,39,341,SINGLEBACK,11.0,3041
2615,3.0,7,4,38,637,SHOTGUN,6.0,637
11424,3.0,20,2,50,43,SHOTGUN,4.0,1843
...,...,...,...,...,...,...,...,...
13583,1.0,10,1,39,527,SINGLEBACK,8.0,3227
5458,4.0,22,4,26,541,SHOTGUN,6.0,541
872,1.0,10,1,43,385,SINGLEBACK,12.0,3085
15980,2.0,6,4,6,215,SHOTGUN,7.0,215


In [28]:
X_train["offenseFormation"].value_counts()

offenseFormation
SHOTGUN       7039
SINGLEBACK    3103
EMPTY         1085
I_FORM         826
PISTOL         524
JUMBO           98
WILDCAT         73
Name: count, dtype: int64

In [29]:
numerical_features = ["down", "ydstogo", "qtr", "yardlineNumber", "quarter_seconds_remaining", "playClockAtSnap", "total_seconds_remaining"]
categorical_features = ["offenseFormation"]

In [30]:
preprocessor = ColumnTransformer(
    transformers=[
        ("num", MinMaxScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown="ignore", sparse_output=True), categorical_features)
    ]
)

In [31]:
X_train = preprocessor.fit_transform(X_train)

In [32]:
X_train

array([[0.33333333, 0.        , 0.75      , ..., 1.        , 0.        ,
        0.        ],
       [0.33333333, 0.21621622, 0.5       , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.24324324, 0.        , ..., 0.        , 1.        ,
        0.        ],
       ...,
       [0.        , 0.24324324, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.33333333, 0.13513514, 0.75      , ..., 1.        , 0.        ,
        0.        ],
       [0.33333333, 0.24324324, 0.25      , ..., 1.        , 0.        ,
        0.        ]])

# MODELING SH*T HERE FOR TRAIN

In [33]:
y_train.value_counts()

play_type
pass    7475
run     5273
Name: count, dtype: int64

In [34]:
# param_grid = {
#     'n_estimators': [100, 200, 300, 400],
#     'max_depth': [None, 10, 20, 30, 40],
#     'min_samples_split': [2, 5, 10],
#     'min_samples_leaf': [1, 2, 4],
#     'max_features': ['sqrt', 'log2', None],  # Valid options for max_features
#     'bootstrap': [True, False]
# }

param_grid = {
    'n_estimators': np.arange(100, 500, 50),
    'max_depth': np.arange(3, 10, 1),
    'learning_rate': np.linspace(0.01, 0.3, 10),
    'subsample': np.linspace(0.5, 1.0, 5),
    'colsample_bytree': np.linspace(0.5, 1.0, 5),
    'gamma': np.linspace(0, 5, 5),
    'reg_alpha': np.logspace(-3, 1, 5),
    'reg_lambda': np.logspace(-3, 1, 5)
}

In [35]:
xgb = XGBClassifier(objective='binary:logistic', random_state=42)

In [36]:
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
# rf_model.fit(X_train, y_train)

In [37]:
# random_search = RandomizedSearchCV(
#     estimator=rf_model,
#     param_distributions=param_grid,
#     n_iter=10,  
#     cv=5,  
#     verbose=2,  
#     random_state=42,  
#     n_jobs=-1  
# )

random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_grid,
    n_iter=30,  # Number of random samples
    scoring='f1',  # Use F1 score for evaluation
    cv=5,
    verbose=2,
    n_jobs=-1
)

In [38]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

In [39]:
random_search.fit(X_train, y_train_encoded)
#best_rf_model = random_search.best_estimator_

Fitting 5 folds for each of 30 candidates, totalling 150 fits


RandomizedSearchCV(cv=5,
                   estimator=XGBClassifier(base_score=None, booster=None,
                                           callbacks=None,
                                           colsample_bylevel=None,
                                           colsample_bynode=None,
                                           colsample_bytree=None, device=None,
                                           early_stopping_rounds=None,
                                           enable_categorical=False,
                                           eval_metric=None, feature_types=None,
                                           gamma=None, grow_policy=None,
                                           importance_type=None,
                                           interaction_constraints=None,
                                           learning_rate...
                                        'learning_rate': array([0.01      , 0.04222222, 0.07444444, 0.10666667, 0.13888889,
       0.17111111, 0.20333333, 0.23555556, 0.26777778, 0.3       ]),
                                        'max_depth': array([3, 4, 5, 6, 7, 8, 9]),
                                        'n_estimators': array([100, 150, 200, 250, 300, 350, 400, 450]),
                                        'reg_alpha': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]),
                                        'reg_lambda': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]),
                                        'subsample': array([0.5  , 0.625, 0.75 , 0.875, 1.   ])},
                   scoring='f1', verbose=2)

# TEST STUFF

In [40]:
X_test["total_seconds_remaining"] = (4 - X_test["qtr"]) * 900 + X_test["quarter_seconds_remaining"]

In [41]:
X_test = preprocessor.transform(X_test)

In [42]:
X_test

array([[0.        , 0.24324324, 0.25      , ..., 1.        , 0.        ,
        0.        ],
       [0.33333333, 0.13513514, 0.75      , ..., 1.        , 0.        ,
        0.        ],
       [0.66666667, 0.05405405, 0.75      , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.33333333, 0.24324324, 0.25      , ..., 1.        , 0.        ,
        0.        ],
       [0.33333333, 0.08108108, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.24324324, 0.25      , ..., 0.        , 1.        ,
        0.        ]])

In [43]:
y_pred_encoded = random_search.best_estimator_.predict(X_test)
y_pred = label_encoder.inverse_transform(y_pred_encoded)

In [44]:
#y_pred = best_rf_model.predict(X_test)
# y_pred = rf_model.predict(X_test)

In [45]:
y_pred

array(['pass', 'pass', 'pass', ..., 'pass', 'run', 'run'], dtype=object)

In [46]:
accuracy = accuracy_score(y_test, y_pred)

In [47]:
precision = precision_score(y_test, y_pred, pos_label='pass')  # Use 'pass' as the positive class
recall = recall_score(y_test, y_pred, pos_label='pass')

In [48]:
accuracy

0.7252195734002509

In [49]:
precision

0.7520964360587002

In [50]:
recall

0.780739934711643

In [51]:
y_test.value_counts()

play_type
pass    1838
run     1350
Name: count, dtype: int64

# Formation Terminology

Worst case option: From nflverse, use qb_dropback booelan feature as a heuristic for yards between o line and qb. 

SHOTGUN: QB 5-7 yds behind o-line        
SINGLEBACK    
EMPTY         
I_FORM         
PISTOL         
JUMBO           
WILDCAT         